In [1]:
from telnetlib import STATUS
import psycopg2
import pandas as pd

username = 'piacobelli'
password = 'PAiac14-'
host = 'greenplum01.corral.tacc.utexas.edu'
port = 5432
database = 'uthealth'

connection = psycopg2.connect(
    host = host, 
    database = database, 
    user = username, 
    port = port, 
    password = password, 
    keepalives = 1, keepalives_idle = 100
)
connection.autocommit = True

C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
# Create new IQVIA counts table. 

iqviaCountCreate = '''
    DROP TABLE IF EXISTS dev.pi_iqvia_counts; 

    CREATE TABLE dev.pi_iqvia_counts_test (
        year INT, table_name TEXT, row_count INT, pat_count INT, claimno_count INT, 
        derv_claimno_count INT, spc_row_count INT, row_count_diff FLOAT, last_updated DATE
    ); 
'''

with connection.cursor() as cursor:
    cursor.execute(iqviaCountCreate)



In [4]:
# Select row counts and patient counts for all IQVIA tables and insert them into the counts table. 

rowCountQuery = '''
    DELETE FROM dev.pi_iqvia_counts_test; 

    INSERT INTO dev.pi_iqvia_counts_test (table_name, year, row_count, pat_count)
    SELECT * FROM (
        SELECT 
            'enroll2' AS table_name, 
            year, 
            COUNT(*) AS row_count, 
            COUNT(DISTINCT pat_id) AS pat_count   
        FROM iqvia.enroll2
        GROUP BY year
        UNION ALL 
        SELECT 
            'enroll_synth' AS table_name, 
            b.year,
            COUNT(a.*) AS row_count, 
            COUNT(DISTINCT a.pat_id) AS pat_count   
        FROM iqvia.enroll_synth a
        JOIN iqvia.enroll2 b ON a.pat_id = b.pat_id
        GROUP BY b.year
        UNION ALL
        SELECT 
            'claims' AS table_name, 
            year, 
            COUNT(*) AS row_count, 
            COUNT(DISTINCT pat_id) AS pat_count
        FROM iqvia.claims
        GROUP BY year
    ) as aggregated_results
    ORDER BY table_name; 
'''

with connection.cursor() as cursor: 
    cursor.execute(rowCountQuery)



In [5]:
# Select claimno from the IQVIA claims table and insert them into the counts table.

claimnoQuery = '''
    UPDATE dev.pi_iqvia_counts_test
    SET claimno_count = aggregated_claimnos.claimno_count
    FROM (SELECT 
            'claims' as table_name, 
            year, 
            COUNT(DISTINCT claimno) as claimno_count
        FROM iqvia.claims
        GROUP BY year
    ) AS aggregated_claimnos
    WHERE dev.pi_iqvia_counts_test.year = aggregated_claimnos.year
    AND dev.pi_iqvia_counts_test.table_name = aggregated_claimnos.table_name;
'''


with connection.cursor() as cursor: 
    cursor.execute(claimnoQuery)

# Check current IQVIA counts table.

with connection.cursor() as cursor: 
    cursor.execute("SELECT * FROM dev.pi_iqvia_counts_test;")
    checkResult = cursor.fetchall()
    columnNames = [desc[0] for desc in cursor.description]
    countDF = pd.DataFrame(checkResult, columns=columnNames)
    print(countDF)

    year    table_name   row_count  pat_count  claimno_count  \
0   2018       enroll2   131127502   13520651            NaN   
1   2018  enroll_synth    74763738    6938959            NaN   
2   2018        claims   431490256   11082298    176199840.0   
3   2008       enroll2   373065024   37240842            NaN   
4   2008  enroll_synth    26232297    2497652            NaN   
5   2008        claims  1040200923   28306345    354021797.0   
6   2022       enroll2   100711716   10776743            NaN   
7   2022  enroll_synth   100711716   10776743            NaN   
8   2022        claims   396047992    8980171    227150251.0   
9   2021       enroll2   102927862   10463206            NaN   
10  2021  enroll_synth   102927862   10463206            NaN   
11  2021        claims   388894022    9037596    223403264.0   
12  2020       enroll2   108365461   11953605            NaN   
13  2020  enroll_synth   108365461   11953605            NaN   
14  2020        claims   350379456    95

In [6]:
# Update last updated date. 

updateData = '''
    UPDATE dev.pi_iqvia_counts_test
    SET last_updated = CURRENT_DATE; 
'''

with connection.cursor() as cursor: 
    cursor.execute(updateData)

In [7]:
# Migrate data to qa_reporting table. 

iqviaReportingCountCreate = '''
    DROP TABLE IF EXISTS qa_reporting.iqvia_counts; 

    CREATE TABLE qa_reporting.iqvia_counts (
        year INT, table_name TEXT, row_count INT, pat_count INT, claimno_count INT, 
        derv_claimno_count INT, spc_row_count INT, row_count_diff FLOAT, last_updated DATE
    ); 
'''

iqviaReportingCountInsert = '''
    INSERT INTO qa_reporting.iqvia_counts (year, table_name, row_count, pat_count, claimno_count, last_updated)
    SELECT * FROM (
        SELECT year, table_name, row_count, pat_count, claimno_count, last_updated
        FROM dev.pi_iqvia_counts_test 
        ORDER BY table_name, year
    ) AS count_results
    ORDER BY table_name;
'''

finalQueries = [iqviaReportingCountCreate, iqviaReportingCountInsert]
with connection.cursor() as cursor: 
    for query in finalQueries: 
        cursor.execute(query)
    